# Comparison Notebook

In [ ]:
import pickle
import pandas as pd
import numpy as np
from tools.Unlearner import Unlearner
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
from torchvision import models
import numpy as np
from types import ClassMethodDescriptorType
from torchvision.datasets.inaturalist import DATASET_MD5
import time
import seaborn as sns
import comparison

## IMPORTANT
The function  comparison.metrics(data,i,j) uses always the file Unlearner which contians the Stochastic Teacher algorithm. So in order to use other algorithms, we need to make a change there and make it more flexible

# CIFAR 10

### Import Results File

Import File with results if you already have them

In [80]:
# Specify the file path where the PKL file is located
file_path = 'cifar_10.pkl'  # Replace with the path to your PKL file

# Open the file in binary read mode
with open(file_path, 'rb') as file:
    cifar_10 = pickle.load(file)

# Now, loaded_dict contains the dictionary loaded from the PKL file
print(cifar_10)

{('CIFAR10', 'cifar10_vgg16_bn', 1): {'forget_unlearn_model': 0.0, 'forget_original_model': 1.0, 'retain_unlearn_model': 0.1111111111111111, 'retain_original_model': 0.9999777777777777, 'test_unlearn_model': 0.1, 'test_original_model': 0.9416, 'unlearn_model': 687.8756194114685}, ('CIFAR10', 'cifar10_vgg19_bn', 1): {'forget_unlearn_model': 0.0, 'forget_original_model': 1.0, 'retain_unlearn_model': 0.1111111111111111, 'retain_original_model': 0.9999555555555556, 'test_unlearn_model': 0.1, 'test_original_model': 0.9391, 'unlearn_model_time': 846.52863073349}, ('CIFAR10', 'cifar10_resnet44', 1): {'forget_unlearn_model': 0.0, 'forget_original_model': 1.0, 'retain_unlearn_model': 0.9763777777777778, 'retain_original_model': 0.9998888888888889, 'test_unlearn_model': 0.7476, 'test_original_model': 0.9401, 'unlearn_model_time': 850.3469896316528}, ('CIFAR10', 'cifar10_resnet56', 1): {'forget_unlearn_model': 0.0, 'forget_original_model': 1.0, 'retain_unlearn_model': 0.9762222222222222, 'retain_

This is the code to run the different combinations of models, labels and subsets of the data on CIFAR 10.

In [ ]:
## CIFAR10

models = {"CIFAR10": ['cifar10_vgg16_bn','cifar10_vgg19_bn', 'cifar10_resnet44','cifar10_resnet56']}
classes = [[1],[1,2]]
cifar_10 = {}
for data,model in models.items():
  for i in model:
    for j in classes:
      cifar_10[data,i,len(j)] = comparison.metrics(data,i,j)
cifar_10

### Export Results File

In [75]:
# Specify the file path
file_path = 'cifar_100.pkl'

# Open the file in binary write mode
with open(file_path, 'wb') as file:
    pickle.dump(cifar_100, file)

print("Dictionary saved to", file_path)

Dictionary saved to cifar_100.pkl


### Show results

In [81]:
# Create a MultiIndex for rows and columns
rows = pd.MultiIndex.from_tuples([('cifar10_vgg16_bn', '1'), ('cifar10_vgg16_bn', '2'), ('cifar10_vgg19_bn', '1'), ('cifar10_vgg19_bn', '2'),
                                  ('cifar10_resnet44', '1'), ('cifar10_resnet44', '2'), ('cifar10_resnet56', '1'), ('cifar10_resnet56', '2')], names=['Model', 'Labels'])
columns = pd.MultiIndex.from_tuples([('Forget', 'Unlearned'),('Forget', 'Original'), ('Retain', 'Unlearned'),('Retain', 'Original'),('Test', 'Unlearned'),('Test', 'Original')], names=['Dataset', 'Model'])

data = np.empty((8, 6))
for j,z in enumerate(cifar_10.values()):
  for i,v in enumerate(list(z.values())[0:6]):
    data[j,i] = v
# Create the DataFrame
df = pd.DataFrame(data, index=rows, columns=columns)


# Function to center-align text
def center_align(val):
    return 'text-align: center'
def format_index(label):
    return f'[{label}]'

# Apply styling to format and center-align the MultiIndex labels
styled_df = df.rename_axis(index=format_index).style.set_table_styles(
    [{'selector': 'th', 'props': [('text-align', 'center')]}], axis=0)
# Display the styled DataFrame


styled_df

# CIFAR 100

### Import Results

Import File with results if you already have them

In [82]:
# Specify the file path where the PKL file is located
file_path = 'cifar_100.pkl'  # Replace with the path to your PKL file

# Open the file in binary read mode
with open(file_path, 'rb') as file:
    cifar_100 = pickle.load(file)

# Now, loaded_dict contains the dictionary loaded from the PKL file
print(cifar_100)

{('CIFAR100', 'cifar100_vgg16_bn', 1): {'unlearn_model_forget_dataloader': 0.0, 'model_forget_dataloader': 1.0, 'unlearn_model_retain_dataloader': 0.010101010101010102, 'model_retain_dataloader': 0.9995353535353535, 'unlearn_model_testloader': 0.01, 'model_testloader': 0.7401, 'unlearn_model_time': 118.30202317237854}, ('CIFAR100', 'cifar100_vgg19_bn', 1): {'unlearn_model_forget_dataloader': 0.0, 'model_forget_dataloader': 1.0, 'unlearn_model_retain_dataloader': 0.010101010101010102, 'model_retain_dataloader': 0.9994545454545455, 'unlearn_model_testloader': 0.01, 'model_testloader': 0.7386, 'unlearn_model_time': 146.33853697776794}, ('CIFAR100', 'cifar100_resnet44', 1): {'unlearn_model_forget_dataloader': 0.0, 'model_forget_dataloader': 1.0, 'unlearn_model_retain_dataloader': 0.4854141414141414, 'model_retain_dataloader': 0.9968282828282828, 'unlearn_model_testloader': 0.4458, 'model_testloader': 0.7164, 'unlearn_model_time': 151.8172435760498}, ('CIFAR100', 'cifar100_resnet56', 1): {'

This is the code to run the different models on CIFAR 100.


In [ ]:
## CIFAR100

models = {"CIFAR100": ['cifar100_vgg16_bn','cifar100_vgg19_bn', 'cifar100_resnet44','cifar100_resnet56']}
classes = [[1],[1,2]]
cifar_100 = {}
for data,model in models.items():
  for i in model:
    for j in classes:
      cifar_100[data,i,len(j)] = comparison.metrics(data,i,j)
cifar_100

### Export Results

In [ ]:
# Specify the file path
file_path = 'cifar_100.pkl'

# Open the file in binary write mode
with open(file_path, 'wb') as file:
    pickle.dump(cifar_100, file)

print("Dictionary saved to", file_path)

### Show Results

In [83]:
# Create a MultiIndex for rows and columns
rows = pd.MultiIndex.from_tuples([('cifar100_vgg16_bn', '1'), ('cifar100_vgg16_bn', '2'), ('cifar100_vgg19_bn', '1'), ('cifar100_vgg19_bn', '2'),
                                  ('cifar100_resnet44', '1'), ('cifar100_resnet44', '2'), ('cifar100_resnet56', '1'), ('cifar100_resnet56', '2')], names=['Model', 'Labels'])
columns = pd.MultiIndex.from_tuples([('Forget', 'Unlearned'),('Forget', 'Original'), ('Retain', 'Unlearned'),('Retain', 'Original'),('Test', 'Unlearned'),('Test', 'Original')], names=['Dataset', 'Model'])

data = np.empty((8, 6))
for j,z in enumerate(cifar_100.values()):
  for i,v in enumerate(list(z.values())[0:6]):
    data[j,i] = v
# Create the DataFrame
df = pd.DataFrame(data, index=rows, columns=columns)


# Function to center-align text
def center_align(val):
    return 'text-align: center'
def format_index(label):
    return f'[{label}]'

# Apply styling to format and center-align the MultiIndex labels
styled_df = df.rename_axis(index=format_index).style.set_table_styles(
    [{'selector': 'th', 'props': [('text-align', 'center')]}], axis=0)
# Display the styled DataFrame


styled_df
